In [2]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import numpy as np
from pytube import YouTube

In [3]:
# Load csv into a dataframe
data = pd.read_csv('13-07-2022-14_59_00_master.csv')   
df = pd.DataFrame(data)

In [4]:
# Clean data
df['clarity_level'] = df['clarity_level'].fillna('none') # fill NaN values with 'none'
df['bad_egg'] = df['bad_egg'].astype(str) # change bad_egg column from boolean to string
df['clarity_level'] = np.where(df['bad_egg'] == 'True', df['bad_egg'], df['clarity_level']) # combine bad_egg and clarity_level where bad_egg = True
df['clarity_level'] = df['clarity_level'].replace(['True'],'bad') # change value of bad_egg from 'True' to 'bad'
df = df.drop(['bad_egg', 'split'], axis=1) # drop bad_egg and split columns
df.head()

,label,youtube_id,time_start,time_end,clarity_level
0,crowd,4pRPc04xoEo1,0,12,none
1,spray,4pRPc04xoEo2,12,18,hard
2,running,4pRPc04xoEo3,12,21,hard
3,crowd,4pRPc04xoEo4,34,44,none
4,crowd,4pRPc04xoEo5,66,71,none


In [5]:
urls = []

for ind, row in df.iterrows():

    id = str(row['youtube_id'])
    url = f'https://www.youtube.com/watch?v={id[0:11]}'
    urls.append(url)
    #yt = YouTube(url)
    #video_name = yt.streams[0].title
    #video_length = yt.length
    
unique_urls = set(urls)




In [ ]:
#establishing the connection
conn = psycopg2.connect(
   database="var_data", user='postgres', password='Zestymeatloaf1', host='localhost', port= '5432'
)
conn.autocommit = True

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

In [ ]:
def insert_into_table(cursor, youtube_id, url):
    insert_into_var_data = ("""INSERT INTO video_urls (youtube_id, url)
    VALUES(%s,%s);""")
    row_to_insert = (youtube_id, url)
    cursor.execute(insert_into_var_data, row_to_insert)

In [ ]:
for url in unique_urls:
    youtube_id = url[32:]
    insert_into_table(cursor, youtube_id, url)

In [ ]:
# Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()

In [ ]:
for url in unique_urls:

    youtube_id = url[32:]
    #yt = YouTube(url)
    #video_name = yt.streams[0].title
    #video_length = yt.length
    #print(url, youtube_id)
    
    '''except:
        print('video unavailable')
        continue'''